In [2]:
%logstop
%logstart -rtq ~/.logs/ml.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# ML Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to create effective machine learning models for making predictions. We will be working with nursing home inspection data from the United States, predicting which providers may be fined and for how much.

## Scoring

In this miniproject you will often submit your model's `predict` or `predict_proba` method to the grader. The grader will assess the performance of your model using a scoring metric, comparing it against the score of a reference model. We will use the [average precision score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). If your model performs better than the reference solution, then you can score higher than 1.0.

**Note:** If you use an estimator that relies on random draws (like a `RandomForestClassifier`) you should set the `random_state=` to an integer so that your results are reproducible. 

## Downloading the data

We can download the data set from Amazon S3:

In [4]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -nc -P ./ml-data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -nc -P ./ml-data

mkdir: cannot create directory ‘data’: File exists
File ‘./ml-data/providers-train.csv’ already there; not retrieving.

File ‘./ml-data/providers-metadata.csv’ already there; not retrieving.



We'll load the data into a Pandas DataFrame. Several columns will become target labels in future questions. Let's pop those columns out from the data, and drop related columns that are neither targets nor reasonable features (i.e. we don't wouldn't know how many times a facility denied payment before knowing whether it was fined).

The data has many columns. We have also provided a data dictionary.

In [5]:
import numpy as np
import pandas as pd

In [6]:
metadata = pd.read_csv('./ml-data/providers-metadata.csv')
metadata.head()

Variable                    Label              Description  \
0   PROVNUM  Federal Provider Number  Federal Provider Number   
1  PROVNAME            Provider Name            Provider Name   
2   ADDRESS         Provider Address         Provider Address   
3      CITY            Provider City            Provider City   
4     STATE           Provider State           Provider State   

                            Format  
0        6 alphanumeric characters  
1                             text  
2                             text  
3                             text  
4  2-character postal abbreviation

In [7]:
data = pd.read_csv('./ml-data/providers-train.csv', encoding='latin1')

fine_counts = data.pop('FINE_CNT')
fine_totals = data.pop('FINE_TOT')
cycle_2_score = data.pop('CYCLE_2_TOTAL_SCORE')

## Question 1: state_model

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, `StateMeanEstimator`, below.

In [8]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, grouper):
        self.grouper = grouper
        self.group_averages = {}
        self.global_mean = 0

    def fit(self, X, y):
        self.group_averages = y.groupby(X[self.grouper]).mean().to_dict()
        self.global_mean = y.mean()
        return self

    def predict(self, X):
        if not isinstance(X,pd.DataFrame):
            X = pd.DataFrame(X)
        return [self.group_averages.get(row, self.global_mean)
                for row in X[self.grouper]]

After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [9]:
from sklearn.pipeline import Pipeline

state_model = Pipeline([
    ('sme', GroupMeanEstimator(grouper='STATE'))
    ])
state_model.fit(data, fine_totals)

Pipeline(memory=None, steps=[('sme', GroupMeanEstimator(grouper='STATE'))],
         verbose=False)

Next we should test that our predict method works.

In [10]:
state_model.predict(data.sample(5))

[16722.25925925926,
 560.171052631579,
 8214.822977725675,
 6634.197226502311,
 18147.119744058502]

However, what if we have data from a nursing home in a state (or territory) of the US which is not in the training data?

In [11]:
state_model.predict(pd.DataFrame([{'STATE': 'AS'}]))

[14969.857687877915]

Make sure your model can handle this possibility before submitting your model's predict method to the grader.

In [12]:
grader.score.ml__state_model(state_model.predict)

Your score:  0.9999999999999999


## Question 2: simple_features_model

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a `Pipeline` containing transformers that process these features, followed by an estimator.

**Note:** When the grader checks your answer, it passes a list of dictionaries to the `predict` or `predict_proba` method of your estimator, not a DataFrame. This means that your model must work with both data types. For this reason, we've provided a custom `ColumnSelectTransformer` for you to use instead `scikit-learn`'s own `ColumnTransformer`.

In [13]:
X = data
y = fine_counts > 0

In [14]:
print(len(X.columns))
simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']
print(len(simple_cols))

29
9


In [15]:
X[simple_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13892 entries, 0 to 13891
Data columns (total 9 columns):
BEDCERT             13892 non-null int64
RESTOT              13483 non-null float64
INHOSP              13892 non-null bool
CCRC_FACIL          13892 non-null bool
SFF                 13892 non-null bool
CHOW_LAST_12MOS     13892 non-null bool
SPRINKLER_STATUS    13892 non-null bool
EXP_TOTAL           13104 non-null float64
ADJ_TOTAL           13104 non-null float64
dtypes: bool(5), float64(3), int64(1)
memory usage: 502.1 KB


In [16]:
from sklearn.impute import SimpleImputer

simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return X[self.columns]
        
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
    ('imputer', SimpleImputer())
])

**Note:** The assertion below assumes the output of `noncategorical_features.fit_transform` is a `ndarray`, not a `DataFrame`.)

In [17]:
assert data['RESTOT'].isnull().sum() > 0
assert not np.isnan(simple_features.fit_transform(data)).any()

Now combine the `simple_features` pipeline with an estimator in a new pipeline. Fit `simple_features_model` to the data and submit `simple_features_model.predict_proba` to the grader. You may wish to use cross-validation to tune the hyperparameters of your model.

In [18]:
type(simple_features)

sklearn.pipeline.Pipeline

In [19]:
pd.DataFrame(simple_features.fit_transform(data)).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13892 entries, 0 to 13891
Data columns (total 9 columns):
0    13892 non-null float64
1    13892 non-null float64
2    13892 non-null float64
3    13892 non-null float64
4    13892 non-null float64
5    13892 non-null float64
6    13892 non-null float64
7    13892 non-null float64
8    13892 non-null float64
dtypes: float64(9)
memory usage: 976.9 KB


In [20]:
X[simple_cols].head()

BEDCERT  RESTOT  INHOSP  CCRC_FACIL    SFF  CHOW_LAST_12MOS  \
0       85    74.2    True       False  False            False   
1       50     NaN    True       False  False            False   
2       92    79.8   False       False  False            False   
3      103    98.1   False        True  False            False   
4      149   119.7   False       False  False            False   

   SPRINKLER_STATUS  EXP_TOTAL  ADJ_TOTAL  
0              True        NaN        NaN  
1              True        NaN        NaN  
2              True    3.08015    3.83026  
3              True    2.83938    3.95709  
4              True    3.15554    4.07866

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np

param_grid = {'C': np.arange(0.1, 1, 0.2)}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, n_jobs=3, verbose=1)

simple_features_model = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
    ('imputer', SimpleImputer()),
    ('classifier', grid_search)
])

In [22]:
simple_features_model.fit(data, fine_counts > 0)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  25 out of  25 | elapsed:    4.1s finished
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('cst',
                 ColumnSelectTransformer(columns=['BEDCERT', 'RESTOT', 'INHOSP',
                                                  'CCRC_FACIL', 'SFF',
                                                  'CHOW_LAST_12MOS',
                                                  'SPRINKLER_STATUS',
                                                  'EXP_TOTAL', 'ADJ_TOTAL'])),
                ('imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('classifier',
                 GridSearchCV(cv=5, error_score='raise-de...
                                                           dual=False,
                                                           fit_intercept=True,
                                                           intercept_scaling=1,
                                          

In [23]:
def positive_probability(model):
    def predict_proba(X):
        return simple_features_model.predict_proba(X)[:, 1]
    return predict_proba

grader.score.ml__simple_features(positive_probability(simple_features_model))

Your score:  1.0064538165605996


## Question 3: categorical_features

The `'OWNERSHIP'` and `'CERTIFICATION'` columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator. Construct one or more pipelines for this purpose. Transformers such as [LabelEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder) and [OneHotEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) may be useful, but you may also want to define your own transformers.

If you used more than one `Pipeline`, combine them with a `FeatureUnion`. As in Question 2, we will combine this with an estimator, fit it, and submit the `predict_proba` method to the grader.

In [24]:
print(X.OWNERSHIP.nunique())
print(X.CERTIFICATION.nunique())

13
3


In [25]:
X[['OWNERSHIP','CERTIFICATION']].head()

OWNERSHIP          CERTIFICATION
0  For profit - Corporation  Medicare and Medicaid
1       Government - County  Medicare and Medicaid
2  For profit - Corporation  Medicare and Medicaid
3  For profit - Corporation  Medicare and Medicaid
4  For profit - Corporation  Medicare and Medicaid

In [26]:
X.OWNERSHIP.value_counts()

For profit - Corporation                  8091
Non profit - Corporation                  2497
For profit - Partnership                   890
For profit - Individual                    646
Non profit - Church related                493
Government - County                        429
Non profit - Other                         275
Government - Hospital district             227
Government - State                         142
Government - City/county                    77
Government - City                           71
For profit - Limited Liability company      43
Government - Federal                        11
Name: OWNERSHIP, dtype: int64

In [27]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

owner_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['OWNERSHIP'])),
    ('ohe', OneHotEncoder(categories='auto', sparse=False))
])

cert_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['CERTIFICATION'])),
    ('ohe', OneHotEncoder(categories='auto', sparse=False))
])

categorical_features = FeatureUnion([
    ('owner', owner_onehot),
    ('cert', cert_onehot)
])

In [28]:
assert categorical_features.fit_transform(data).shape[0] == data.shape[0]
assert categorical_features.fit_transform(data).dtype == np.float64
assert not np.isnan(categorical_features.fit_transform(data)).any()

As in the previous question, create a model using the `categorical_features`, fit it to the data, and submit its `predict_proba` method to the grader.

In [29]:
X[['OWNERSHIP', 'CERTIFICATION']].shape

(13892, 2)

In [30]:
X.trans = categorical_features.fit_transform(X)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [32]:
from sklearn.naive_bayes import MultinomialNB

In [33]:
categorical_features_model = Pipeline([
    ('categorical', categorical_features),
  ('classifier', MultinomialNB(alpha=0.01))
])

In [34]:
categorical_features_model.fit(data, fine_counts > 0)

Pipeline(memory=None,
         steps=[('categorical',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('owner',
                                                 Pipeline(memory=None,
                                                          steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['OWNERSHIP'])),
                                                                 ('ohe',
                                                                  OneHotEncoder(categorical_features=None,
                                                                                categories='auto',
                                                                                drop=None,
                                                                                dtype=<class 'numpy.float64'>,
                                                                                handle_unknown='error',
 

In [35]:
def positive_probability(model):
    def predict_proba(X):
        return model.predict_proba(X)[:, 1]
    return predict_proba


grader.score.ml__categorical_features(positive_probability(categorical_features_model))

Your score:  0.9747736478437008


## Question 4: business_model

Finally, we'll combine `simple_features` and `categorical_features` in a `FeatureUnion`, followed by an estimator in a `Pipeline`. You may want to optimize the hyperparameters of your estimator using cross-validation or try engineering new features (e.g. see [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)). When you've assembled and trained your model, pass the `predict_proba` method to the grader.

In [36]:
business_features = FeatureUnion([
    ('simple', simple_features),
    ('categorical', categorical_features)
])

In [37]:
from sklearn.preprocessing import PolynomialFeatures

business_model = Pipeline([
    ('features', business_features),
    ('poly', PolynomialFeatures(2)),
    ('classifier', LogisticRegression())
])

In [38]:
business_model.fit(data, fine_counts > 0);

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [39]:
business_model.score(data, fine_counts > 0)

0.6970198675496688

In [40]:
grader.score.ml__business_model(positive_probability(business_model))

Error!
You have been rate limited for exceeding the limit of 3 per 1 minute.
Please slow down your submission rate.


## Question 5: survey_results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the `business_features` of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to predict the cycle 2 total score.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [84]:
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X,pd.DataFrame):
            X = pd.DataFrame(X)
        results = (pd.to_datetime(X[self.t1_col]) 
                   - pd.to_datetime(X[self.t2_col]))
        results = results.apply(lambda x: x.days).values.reshape(-1,1)
        return results

In [85]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date, cycle_2_date)

In the cell below we'll collect the cycle 1 survey features.

In [86]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']
cycle_1_features = Pipeline([
    ('cst2', ColumnSelectTransformer(cycle_1_cols)),
    ])

In [87]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import Lasso

gs = GridSearchCV(Lasso(max_iter = 1000), param_grid = {'alpha': np.arange(0, 3.5, 0.5)}, cv=5, n_jobs=4, verbose=1)


survey_model = Pipeline([
    ('features', FeatureUnion([
        ('business', business_features),
        ('survey', cycle_1_features),
        ('time', time_feature)
    ])),
    # add your estimator here
    ('poly', PolynomialFeatures(2)),
    ('svd', TruncatedSVD(40)),
    ('gs', gs)
])

In [88]:
survey_model.fit(data, cycle_2_score.astype(int))

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  35 out of  35 | elapsed:    7.4s finished


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('business',
                                                 FeatureUnion(n_jobs=None,
                                                              transformer_list=[('simple',
                                                                                 Pipeline(memory=None,
                                                                                          steps=[('cst',
                                                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                                                   'RESTOT',
                                                                                                                                   'INHOSP',
                                               

In [89]:
grader.score.ml__survey_model(survey_model.predict)

Your score:  1.1641229173905947


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*